In [1]:
%load_ext autoreload
%autoreload 2

from src.data.mnist import MNISTDataModule
from torch import nn

import torch
import torch.nn.functional as F
import pytorch_lightning as pl

import torchmetrics

In [2]:
dm = MNISTDataModule(batch_size=16)

First, a baseline model for comparison

In [1]:
class BaselineModel(pl.LightningModule):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

        self.accuracy = torchmetrics.Accuracy()

    def forward(self, x):

        x = self.model(x.view(-1, 784))
        return x

    def training_step(self, batch, batch_idx):

        x, y = batch
        y_hat = self.model(x.view(-1, 784))
        loss = F.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):

        x, y = batch
        y_hat = self.model(x.view(-1, 784))
        loss = F.cross_entropy(y_hat, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log('val_acc_step', self.accuracy(y_hat, y))

        return loss

    def validation_epoch_end(self, outs):

        self.log("val_acc_epoch", self.accuracy.compute(), prog_bar=True)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

baseline_model = BaselineModel()


NameError: name 'pl' is not defined

In [7]:
trainer = pl.Trainer(max_epochs=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
trainer.fit(model=baseline_model, datamodule=dm)

/Users/soren/Repositories/Speciale/.venv/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name     | Type       | Params
----------------------------------------
0 | model    | Sequential | 136 K 
1 | accuracy | Accuracy   | 0     
----------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Epoch 4: 100%|██████████| 3751/3751 [00:30<00:00, 121.67it/s, loss=0.196, v_num=6, val_loss=0.247, val_acc_step=0.947, val_acc_epoch=0.937]


Generative process:

1. Draw $\bm \theta \sim \mathcal N (0, \alpha^2 I)$
2. Draw $\bm y \sim Dist_y(NN(x;\theta))$ (For whatever observation model, in this case Dirichlet)


$$p(\bm y, \theta | \alpha) = p(\theta|0, \alpha^2 I) \prod_{n=1}^N p(y_n | \theta, X) $$ 



In [91]:
class Model(nn.Module):
    
    def __init__(self, alpha=0.1):
        
        super().__init__()
        self.alpha = alpha
        self.ffnn = nn.Sequential(
                nn.Linear(784, 128),
                nn.ReLU(),
                nn.Linear(128, 256),
                nn.ReLU(),
                nn.Linear(256, 10),
            )

    def posterior(self, x):

        x = self.ffnn(x.view(-1, 784))
        return torch.distributions.Categorical(x.exp())

    def log_joint_prob(self, batch):

        x, y = batch

        lp = sum(param_prior.log_prob(x).sum() for x in self.parameters())
        lp +=self.posterior(x).log_prob(y).sum()

        return lp

model = Model()
batch = next(iter(dm.train_dataloader()))


In [94]:
model.log_joint_prob(batch)

tensor(181635.7656, grad_fn=<AddBackward0>)

In [93]:
posterior.log_prob(y)

tensor([-2.2923, -2.3584, -2.2544, -2.3829, -2.2565, -2.3082, -2.2628, -2.2470,
        -2.2318, -2.3020, -2.2992, -2.2804, -2.3048, -2.2823, -2.3020, -2.2854],
       grad_fn=<SqueezeBackward1>)

In [81]:
y

tensor([5, 6, 4, 9, 5, 2, 8, 5, 5, 7, 4, 3, 1, 3, 2, 7])

In [80]:
param_prior = torch.distributions.Normal(0, 0.1)

# computes log joint probability of the model (y, X, lambda and sigma are assume to be fixed/given)
def log_joint_prob():
    # log probability of the prior
    
    
    # log likelihood
    res = model(x)
        
    return lp

# a few tests checks
log_joint_prob(parameters)

TypeError: log_joint_prob() takes 0 positional arguments but 1 was given